In [1]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
from src.model import WrapHookedTransformer
from src.dataset import TlensDataset
from src.experiment import LogitAttribution
%load_ext autoreload
%autoreload 2

In [2]:
model = WrapHookedTransformer.from_pretrained("gpt2")
dataset = TlensDataset("../data/full_data_sampled_gpt2.json", model, slice=100)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer
Dataset loaded from ../data/full_data_sampled_gpt2.json
Number of samples: 100


In [3]:
logit, cache = model.run_with_cache("ciao come va?")

In [42]:
matrix = (model.W_U.T @ (model.OV[0,0] @ model.W_E_pos.T))

In [5]:
from src.experiment import OV
ov = OV(dataset, model, 20)

In [7]:
batch = ov.get_batch(21)

In [15]:
token_embedding = model.blocks[0].mlp(token_embedding)

In [20]:
((model.W_U.T @ model.OV[0,0]).AB @ token_embedding.transpose(0,1)) 

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_53670/667625610.py:1 in <module>                                           │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_53670/667625610.py'                  │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: mat1 and mat2 shapes cannot be multiplied (1536x20 and 768x50257)

In [22]:
(model.W_U.T @ model.OV[0,0]).AB.shape

torch.Size([50257, 768])

In [9]:
targets = batch["target"]

In [30]:

(model.W_E[1] + model.W_pos[18]).shape

torch.Size([768])

In [34]:
import torch
target = torch.tensor([[0,1], [0,0], [1,0]])

In [56]:
matrix.shape

torch.Size([50257, 51281])

In [35]:
target.shape

torch.Size([3, 2])

In [57]:
#one-hot encoding the target
target_mem = torch.zeros(target.shape[0], 2, 51281)

In [60]:
target_mem.shape

torch.Size([3, 2, 51281])

In [61]:
target_mem = target_mem.scatter(2, target.unsqueeze(2), 1)

In [ ]:
t

In [65]:
target.unsqueeze(-1).shape

torch.Size([3, 2, 1])

In [66]:
target_mem

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]])

In [69]:
target_mem[:,0,:].shape

torch.Size([3, 51281])

In [70]:
torch.matmul(matrix, target_mem[:,0,:])

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_28244/1006560541.py:1 in <module>                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_28244/1006560541.py'                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: matmul(): argument 'input' (position 1) must be Tensor, not FactoredMatrix

In [98]:
(matrix @ target_mem[:,0,:].T.cuda()).AB.shape

torch.Size([50257, 3])

In [94]:
matrix.shape @ 

torch.Size([50257, 51281])

In [46]:
target.shape

torch.Size([3, 2])

In [47]:

logit = einops.einsum(matrix, target, "out_voc in_voc, batch_size n_target -> batch_size out_voc n_target")


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_28244/796323416.py:1 in <module>                                           │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_28244/796323416.py'                  │
│ /home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/einops/einops.py:793 in │
│ einsum                                                                                    │
│                                                                                           │
│   790 │   │   )                                                                           │
│   791 │   tensors = tensors_and_pattern[:-1]                                              │
│   792 │   pattern = _compactify_pattern_for_einsum(pattern)                               │
│ ❱ 793 │   return get_backend(tensors[0]).einsum(pattern, *tensors)                        │
│   794                                                                                     │
│                                                                                           │
│ /home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/einops/_backends.py:52  │
│ in get_backend                                                                            │
│                                                                                           │
│    49 │   │   │   │   if backend.is_appropriate_type(tensor):                             │
│    50 │   │   │   │   │   return backend                                                  │
│    51 │                                                                                   │
│ ❱  52 │   raise RuntimeError('Tensor type unknown to einops {}'.format(type(tensor)))     │
│    53                                                                                     │
│    54                                                                                     │
│    55 class AbstractBackend:                                                              │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Tensor type unknown to einops <class 
'transformer_lens.FactoredMatrix.FactoredMatrix'>

In [102]:
torch.tensor([[1], [0], [0]]).shape

torch.Size([3, 1])